In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from sklearn.preprocessing import MinMaxScaler, StandardScaler


#Feature Selection
from sklearn.feature_selection import SelectKBest, RFE

#Classification
from sklearn.svm import SVC

# Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor


# Cross-Validation
from sklearn.model_selection import GroupKFold

from biopsykit.classification.model_selection import SklearnPipelinePermuter


%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
save_results = True

In [3]:
data_path = Path("../../results/data")
data_path

WindowsPath('../../results/data')

In [4]:
models_path = Path("../../results/models")

In [12]:
input_data = pd.read_csv(data_path.joinpath("train_data_no_outlier_correction.csv"), index_col=[0,1,2,3,4])
input_data

arbol2017-isoelectric-crossings  \
      participant phase         heartbeat_id_reference b_point_sample_reference                                    
0     GDN0005     HoldingBreath 0                      388.0                                               438.0   
1     GDN0005     HoldingBreath 1                      404.0                                               340.0   
2     GDN0005     HoldingBreath 3                      376.0                                               382.0   
3     GDN0005     HoldingBreath 4                      390.0                                               394.0   
4     GDN0005     HoldingBreath 5                      386.0                                               398.0   
...                                                                                                          ...   
11219 VP_032      Talk          39                     310.0                                               335.0   
11220 VP_032      Talk          40                     322.0                                               331.0   
11221 VP_032      Talk          41                     340.0                                               317.0   
11222 VP_032      Talk          42                     311.0                                               365.0   
11223 VP_032      Talk          43                     307.0                                               342.0   

                                                                                 arbol2017-second-derivative  \
      participant phase         heartbeat_id_reference b_point_sample_reference                                
0     GDN0005     HoldingBreath 0                      388.0                                           398.0   
1     GDN0005     HoldingBreath 1                      404.0                                           350.0   
2     GDN0005     HoldingBreath 3                      376.0                                           296.0   
3     GDN0005     HoldingBreath 4                      390.0                                           344.0   
4     GDN0005     HoldingBreath 5                      386.0                                           312.0   
...                                                                                                      ...   
11219 VP_032      Talk          39                     310.0                                           276.0   
11220 VP_032      Talk          40                     322.0                                           298.0   
11221 VP_032      Talk          41                     340.0                                           300.0   
11222 VP_032      Talk          42                     311.0                                           324.0   
11223 VP_032      Talk          43                     307.0                                           313.0   

                                                                                 arbol2017-third-derivative  \
      participant phase         heartbeat_id_reference b_point_sample_reference                               
0     GDN0005     HoldingBreath 0                      388.0                                          394.0   
1     GDN0005     HoldingBreath 1                      404.0                                          244.0   
2     GDN0005     HoldingBreath 3                      376.0                                          386.0   
3     GDN0005     HoldingBreath 4                      390.0                                          396.0   
4     GDN0005     HoldingBreath 5                      386.0                                          388.0   
...                                                                                                     ...   
11219 VP_032      Talk          39                     310.0                                          324.0   
11220 VP_032      Talk          40                     322.0                                          330.0   
11221

In [13]:
print(f"Min data value: {input_data.values.min()}\nMax data value: {input_data.values.max()}")

Min data value: -84.0
Max data value: 742.0


In [14]:
columns=input_data.columns
columns

Index(['arbol2017-isoelectric-crossings', 'arbol2017-second-derivative',
       'arbol2017-third-derivative', 'debski1993-second-derivative',
       'drost2022', 'forounzafar2018', 'lozano2007-linear-regression',
       'lozano2007-quadratic-regression', 'sherwood1990', 'stern1985'],
      dtype='object')

In [15]:
negative_data = input_data[input_data.values < 0]
negative_data

arbol2017-isoelectric-crossings  \
      participant phase    heartbeat_id_reference b_point_sample_reference                                    
790   GDN0007     Valsalva 0                      448.0                                               392.0   
6627  VP_002      Math     22                     209.0                                               219.0   
6664  VP_002      Math     45                     209.0                                               224.0   
6666  VP_002      Math     46                     206.0                                               220.0   
6667  VP_002      Math     47                     206.0                                               221.0   
...                                                                                                     ...   
10667 VP_031      Math     34                     201.0                                               175.0   
10669 VP_031      Math     35                     199.0                                               218.0   
10693 VP_031      Math     49                     197.0                                               206.0   
10694 VP_031      Math     50                     197.0                                               220.0   
10699 VP_031      Math     55                     202.0                                               163.0   

                                                                            arbol2017-second-derivative  \
      participant phase    heartbeat_id_reference b_point_sample_reference                                
790   GDN0007     Valsalva 0                      448.0                                           394.0   
6627  VP_002      Math     22                     209.0                                           190.0   
6664  VP_002      Math     45                     209.0                                           184.0   
6666  VP_002      Math     46                     206.0                                           183.0   
6667  VP_002      Math     47                     206.0                                           181.0   
...                                                                                                 ...   
10667 VP_031      Math     34                     201.0                                           158.0   
10669 VP_031      Math     35                     199.0                                           160.0   
10693 VP_031      Math     49                     197.0                                           154.0   
10694 VP_031      Math     50                     197.0                                           151.0   
10699 VP_031      Math     55                     202.0                                           155.0   

                                                                            arbol2017-third-derivative  \
      participant phase    heartbeat_id_reference b_point_sample_reference                               
790   GDN0007     Valsalva 0                      448.0                                          458.0   
6627  VP_002      Math     22                     209.0                                           -9.0   
6664  VP_002      Math     45                     209.0                                          -15.0   
6666  VP_002      Math     46                     206.0                                          -16.0   
6667  VP_002      Math     47                     206.0                                          -18.0   
...                                                                                                ...   
10667 VP_031      Math     34                     201.0                                          -38.0   
10669 VP_031      Math     35                     199.0                                          -34.0   
10693 VP_031      Math     49                     197.0                                          -31.0   
10694 VP_031      Math     50                     197.0                   

In [16]:
X, y, groups, group_keys = bp.classification.utils.prepare_df_sklearn(data=input_data, label_col="b_point_sample_reference", subject_col="participant", print_summary=True)

Shape of X: (10385, 10); shape of y: (10385,); number of groups: 39, class prevalence: [ 2  1  1  4  1  1  1  1  1  2  1  1  2  1  1  2  3  2  6  3  9  4  8  7
 10  4  5 11  7 13  9 11 14 11 15  9 12 15 17 21 22 20 28 27 29 28 34 35
 44 37 37 39 30 34 46 40 50 47 41 38 23 21 26 34 34 27 33 37 26 40 20 26
 33 23 23 29 31 18 28 24 19 29 17 20 24 23 23 11 18 20 16 23 26 24 22 10
 15 14 17 14 17 20 16 13 17 17 25 16 12 19 21 18 19 19 22 17 22 19 21 24
 24 18 27 21 17 23 13 14 22 21 25 12 21 18 27 14 26 17 20 23 19  9 22 16
 24 11 18 21 25 16 31 20 23 25 26 24 27 10 36 15 36 12 36 21 29 16 32 23
 55 16 48 14 38 28 31 11 52  8 56 20 54 23 50 14 53 18 63 18 56 18 55 24
 48 21 59 14 57 11 73 19 44 18 56 12 72 17 56 14 53 16 63 18 77 12 84  9
 60 16 69  9 59  4 66 16 64  7 66  9 82  9 83  6 73  8 71 10 81 10 90  9
 69  7 68  4 74  5 71  3 61  7 78  5 77  2 82  7 73  8 64  8 61  1 81  2
 76  3 63 73  2 68  3 64  2 86  6 66  2 84  2 78  1 66  2 65 72 77  2 56
  1 71 69  1 59  1 76 49 67  1 68 63 

In [17]:
group_keys

Index(['GDN0005', 'GDN0006', 'GDN0007', 'GDN0008', 'GDN0009', 'GDN0010',
       'GDN0011', 'GDN0012', 'GDN0013', 'GDN0014', 'GDN0016', 'GDN0017',
       'GDN0018', 'GDN0019', 'GDN0020', 'GDN0021', 'GDN0022', 'GDN0023',
       'GDN0024', 'GDN0025', 'GDN0027', 'GDN0028', 'GDN0029', 'GDN0030',
       'VP_001', 'VP_002', 'VP_003', 'VP_004', 'VP_005', 'VP_020', 'VP_022',
       'VP_023', 'VP_026', 'VP_027', 'VP_028', 'VP_029', 'VP_030', 'VP_031',
       'VP_032'],
      dtype='object')

In [18]:
model_dict = {
    "scaler": {"StandardScaler": StandardScaler(), "MinMaxScaler": MinMaxScaler()},
    "reduce_dim": {"SelectKBest": SelectKBest(), "RFE": RFE(SVC(kernel="linear"))},
    #"reduce_dim": {"SelectKBest": SelectKBest()},
    "clf": {
        "KNeighborsRegressor": KNeighborsRegressor(),
        #"RandomForestRegressor": RandomForestRegressor(n_jobs=10),
        #"HistGradientBoostingRegressor": HistGradientBoostingRegressor(),
    },
}

In [19]:
params_dict = {
    "StandardScaler": None,
    "MinMaxScaler": None,
    "SelectKBest": {"k": [2, 4, 6, 8, 10, "all"]},
    "SVC": {"C": [0.1, 1, 10]},
    "RFE": {
        "n_features_to_select": [0.4, 0.5, 0.6],
        "step": [1,2,3],
    },
    "KNeighborsRegressor": {
        "n_neighbors": [8,9,10,11,12,13,14],
        "weights": ["uniform", "distance"],
        "p": [1,2],
        },
    #"RandomForestRegressor": {
    #    "n_estimators": [10],
    #    "min_samples_split": [2, 5, 10, 20],
    #    "min_samples_leaf": [1, 2, 4, 10],
    #    "max_depth": [None, 10, 20, 30, 40, 50]
    #    #"max_features": ["sqrt", "log2", None],
    #},
    #"HistGradientBoostingRegressor": None,
}

In [20]:
pipeline_permuter = SklearnPipelinePermuter(
    model_dict=model_dict, param_dict=params_dict
)

In [21]:
outer_cv = GroupKFold(n_splits=5)
inner_cv = GroupKFold(n_splits=5)

pipeline_permuter.fit(X=X, y=y, outer_cv=outer_cv, inner_cv=inner_cv, scoring="neg_mean_absolute_error", groups=groups)

Pipeline Combinations:   0%|          | 0/4 [00:00<?, ?it/s]

### Running hyperparameter search for pipeline: (('scaler', 'StandardScaler'), ('reduce_dim', 'SelectKBest'), ('clf', 'KNeighborsRegressor')) with 1 parameter grid(s):
Parameter grid #0 ({'search_method': 'grid'}): {'reduce_dim__k': [2, 4, 6, 8, 10, 'all'], 'clf__n_neighbors': [8, 9, 10, 11, 12, 13, 14], 'clf__weights': ['uniform', 'distance'], 'clf__p': [1, 2]}


Outer CV:   0%|          | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 168 candidates, totalling 840 fits


C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


...

C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


Fitting 5 folds for each of 252 candidates, totalling 1260 fits


C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


Fitting 5 folds for each of 252 candidates, totalling 1260 fits


C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


In [22]:
pipeline_permuter.metric_summary()

true_labels  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                             
StandardScaler  SelectKBest         KNeighborsRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                RFE                 KNeighborsRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
MinMaxScaler    SelectKBest         KNeighborsRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                RFE                 KNeighborsRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   

                                                                                         true_labels_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                             
StandardScaler  SelectKBest         KNeighborsRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                RFE                 KNeighborsRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
MinMaxScaler    SelectKBest         KNeighborsRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                RFE                 KNeighborsRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   

                                                                                          predicted_labels  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                             
StandardScaler  SelectKBest         KNeighborsRegressor  [357.26026274767827, 342.3397730034228, 366.11...   
                RFE                 KNeighborsRegressor  [363.085485931488, 342.61961720647287, 371.428...   
MinMaxScaler    SelectKBest         KNeighborsRegressor  [359.6414425809928, 341.8356502802455, 366.889...   
                RFE                 KNeighborsRegressor  [371.14289041775334, 346.63798434339765, 377.0...   

                                                                                    predicted_labels_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                             
StandardScaler  SelectKBest         KNeighborsRegressor  [[357.26026274767827, 342.3397730034228, 366.1...   
                RFE                 KNeighborsRegressor  [[363.085485931488, 342.61961720647287, 371.42...   
MinMaxScaler    SelectKBest         KNeighborsRegressor  [[359.6414425809928, 341.8356502802455, 366.88...   
                RFE                 KNeighborsRegressor  [[371.14289041775334, 346.63798434339765, 377....   

                                                                                             train_indices  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                             
StandardScaler  SelectKBest         KNeighborsRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                RFE                 KNeighborsRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
MinMaxScaler    SelectKBest         KNeighborsRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                RFE                 KNeighborsRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   

                                                                                       train_indices_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                             
StandardScaler  SelectKBest         KNeighborsRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   
                RFE                 KNeighborsRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   
MinMaxScaler    SelectKBest         KNeighborsRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   
                RFE                 KNeighborsRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   

                                                                                              test_indices  \
pipeline_scaler pip

In [26]:
pipeline_permuter.best_hyperparameter_pipeline()

mean_test_neg_mean_absolute_error  param_clf__n_neighbors  \
outer_fold                                                              
0                                   -8.635566                      14   
1                                   -8.955911                      14   
2                                   -8.639042                      14   
3                                   -9.138771                      14   
4                                   -8.743441                      14   

            param_clf__p param_clf__weights  \
outer_fold                                    
0                      1           distance   
1                      1           distance   
2                      1           distance   
3                      1           distance   
4                      1           distance   

            param_reduce_dim__n_features_to_select  param_reduce_dim__step  \
outer_fold                                                                   
0                                              0.4                     1.0   
1                                              0.4                     1.0   
2                                              0.4                     1.0   
3                                              0.4                     1.0   
4                                              0.4                     1.0   

                                                       params  \
outer_fold                                                      
0           {'clf__n_neighbors': 14, 'clf__p': 1, 'clf__we...   
1           {'clf__n_neighbors': 14, 'clf__p': 1, 'clf__we...   
2           {'clf__n_neighbors': 14, 'clf__p': 1, 'clf__we...   
3           {'clf__n_neighbors': 14, 'clf__p': 1, 'clf__we...   
4           {'clf__n_neighbors': 14, 'clf__p': 1, 'clf__we...   

            rank_test_neg_mean_absolute_error  \
outer_fold                                      
0                                           6   
1                                           1   
2                                           1   
3                                           4   
4                                           8   

            split0_test_neg_mean_absolute_error  \
outer_fold                                        
0                                     -7.355686   
1                                     -8.266636   
2                                     -8.238201   
3                                     -9.782426   
4                                     -7.631981   

            split1_test_neg_mean_absolute_error  \
outer_fold                                        
0                                     -9.623169   
1                                    -10.614622   
2                                     -9.115226   
3                                     -9.631542   
4                                     -8.958766   

            split2_test_neg_mean_absolute_error  \
outer_fold                                        
0                                     -8.323662   
1                                     -8.423419   
2                                     -9.022021   
3                                     -8.410206   
4                                    -10.848190   

            split3_test_neg_mean_absolute_error  \
outer_fold                                        
0                                     -8.708293   
1                                     -8.599175   
2                                     -8.862420   
3                                     -8.830565   
4                                     -8.984664   

            split4_test_neg_mean_absolute_error  \
outer_fold                                        
0                                     -9.167020   
1                                     -8.875704   
2                                     -7.957339   
3                                     -9.039118   
4                                     -7.293606   

            std_test_neg_mean_abs

In [24]:
print(f"Save results: {save_results}")

Save results: True


In [25]:
if save_results:
    pipeline_permuter.to_pickle(models_path.joinpath("KNeighbors_No_Outlier_Correction.pkl"))